Run this cell if the dataset/variables are not present from running Preprocessing part1 - thresholding

In [6]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
sns.set_style('dark', rc={'image.cmap':'inferno'})

#THE real data to be imported in the class is commented out for now because files are currently too large for github

# data_nodrug = imread("../data/confocal_drug_panel/DMSO.tif")
data_nodrug = imread("../data/confocal_drug_panel/temp_DMSO.tif")

import json
with open('../data/confocal_drug_panel/DMSO_metadata.json', mode='r') as f_nodrug:
    meta_nodrug = json.load(f_nodrug)

nodrug_slice = {}
for idx, channel in enumerate(meta_nodrug['channels']):
    nodrug_slice[channel] = data_nodrug #[4,:,:,idx]    #add in the indexing when read in full dataset

data = nodrug_slice['actin']

In [7]:
#parameters to adjust
minX1 = 700 #crop edges for a cell in the center of field of view
minY1 = 900
minX2 = 1200 #crop edges for cell at the edge of the field of view
minY2 = 1800
crop_size = 200 #pix
image_view_thresh = 0.1

#run
maxX1 = minX1 + crop_size
maxY1 = minY1 + crop_size
maxX2 = minX2 + crop_size
maxY2 = minY2 + crop_size

top = data.max() * image_view_thresh

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[1].imshow(data[minY1 : maxY1 , minX1 : maxX1], vmin=0, vmax=top)
ax[0].imshow(data, vmin=0, vmax=top)
ax[2].imshow(data[minY2 : maxY2, minX2: maxX2], vmin=0, vmax=top)

from skimage import filters
thresh = filters.threshold_otsu(data)
print("the Otsu masking threshold for this dataset is:", thresh)
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
mask = np.zeros(nodrug_slice["actin"].shape)
mask[nodrug_slice["actin"] >=thresh] = 1
mask_zoom_center = mask[minY1 : maxY1 , minX1 : maxX1]
mask_zoom_edge = mask[minY2 : maxY2 , minX2 : maxX2]
ax[0].imshow(mask, vmin=0, vmax=1)
ax[1].imshow(mask_zoom_center, vmin=0, vmax=1)
ax[2].imshow(mask_zoom_edge, vmin=0, vmax=1)

In [ ]:
#flatten the field to show that it improves Otsu's method--show the new histogram
    #Methods: rolling ball, subtract Gaussian blur, large min filter

In [9]:
#show new masks-- they are improved but still have holes and rogue noisy pixels
#removing shot noise: median filtering (shown below)
#morphological processing (From Morphological Image Processing module)


### Removing Shot Noise from your Image -- Median Fitering

To get better at object detection, we can leverage various properties of the pixels. With time, you will be able to leverage pretty much any property you can articulate, but for now, let's use the idea that the pixels that are hanging out in the wrong places are surrounded by other pixels that are properly classified. Let's make them listen to their neighbours. There are many ways to do this. One useful method to know is called Median filtering. It goes pixel by pixel, and replaces each pixel with the median of its surroundings. Let's load our image slice...

In [ ]:
from scipy.ndimage.filters import median_filter
plt.imshow(original_slice)

Tech tip: These images were taken with a confocal microscope, which uses a PMT (photomultiplier tube) with high sensitivity. However, because this detector operates in a low-photon regime, shot noise (Poisson distributed) can add substantial deviation of pixel values from the local fluorescence intensities they represent. Shot noise is commonly removed with the median filter, although other rank filters exist.

In [12]:
from ipywidgets import interactive

@interactive
def apply_filter(size=(1, 21)):
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))
    
    # Here we implement the median filtering
    filtered = median_filter(original_slice, size=size)
                             
    ax[0].imshow(original_slice)
    ax[1].imshow(filtered)
    dif_img = filtered.astype('int') - original_slice.astype('int')
    
    extreme = 10000
    im = ax[2].imshow(dif_img, vmin=-extreme, vmax=extreme, cmap='coolwarm')
    
    print("total difference in image =" + str(np.mean(dif_img)) + " arbitrary units")
    print("percent change =" + str(np.mean(dif_img)/100) + "%") 
apply_filter

Note that the size of the filter determines the value of the median value of the pixels in the output. That means, the larger the filter size, the more neighbours the filter will look at, before deciding what the new pixel value should be. A good rule of thumb when determining an appropriate filter size is that it should be the smallest filter that sufficiently flattens the visible noise in the background. Many of these operations do not have well-accepted statistical tests for determing the appropriate parameters, so care needs to be taken to record and reproduce processing steps with the same parameters. 

Let's choose a filter size of 3x3.

In [14]:
filtered_slice = median_filter(original_slice, size=3)
filtered_image = median_filter(whole_image, size=3)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(filtered_image)
ax[1].imshow(filtered_slice)

Now let's see how the filtering affects our mask, and compare to the mask we made earlier.

In [16]:
masked_filtered_slice = threshold_image(filtered_slice, filters.threshold_otsu(filtered_slice))
masked_filtered_image = threshold_image(filtered_image, filters.threshold_otsu(filtered_image))

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(masked_filtered_image)
ax[1].imshow(masked_slice)
ax[2].imshow(masked_filtered_slice)

Great, but still not perfect! What do you think would happen if we tried to apply the mask before the filter? (this could be an exercise)

In [18]:
filtered_masked_slice = median_filter(masked_slice, size=3)
filtered_masked_image = median_filter(masked_whole_image, size=3)

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(masked_filtered_slice)
ax[1].imshow(filtered_masked_slice)
ax[2].imshow(masked_filtered_slice - filtered_masked_slice, cmap='coolwarm')